# UHG IDS Data Export for Unreal Engine 5
This notebook exports network intrusion detection data and model results for 3D visualization in Unreal Engine 5.


In [ ]:
# Install required packages
!pip -q install --upgrade pip
import torch
pt = torch.__version__.split('+')[0]
cuda = torch.version.cuda
if torch.cuda.is_available() and cuda:
  idx = f"https://data.pyg.org/whl/torch-{pt}+cu{cuda.replace('.','')}.html"
else:
  idx = f"https://data.pyg.org/whl/torch-{pt}+cpu.html"

!pip -q install torch_scatter torch_sparse torch_cluster torch_spline_conv -f {idx}
!pip -q install torch_geometric scikit-learn scipy pandas tqdm


In [ ]:
# Import all necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
from scipy.sparse import coo_matrix
from torch_geometric.data import Data
from torch_geometric.nn.conv import MessagePassing
from torch_scatter import scatter_add
import json
import os
import time
from typing import Tuple, Dict, Any

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {'GPU' if torch.cuda.is_available() else 'CPU'}")

# File paths (update these to match your setup)
FILE_PATH = '/content/drive/MyDrive/CIC_data.csv'
MODEL_SAVE_PATH = '/content/drive/MyDrive/uhg_ids_model.pth'
RESULTS_PATH = '/content/drive/MyDrive/uhg_ids_results'
EXPORT_PATH = '/content/drive/MyDrive/UE5_Export'
os.makedirs(EXPORT_PATH, exist_ok=True)
